In [8]:
import os
import sys
import shutil

import bw2calc as bc
import bw2data as bd
import bw2io as bi

bi.__version__
bd.__version__
bc.__version__

import seaborn as sns

import visualisation as vis
from add_uncertainties import add_uncertainties
from import_db_from_file import write_database, inspect_db, export_db
from LCA_calculations import get_LCA_scores, get_MCLCA_scores, get_LCA_report
from make_process_diagram import extract_nodes_edges, write_process_diagram

In [9]:
bd.projects.set_current('cLCA-aalborg')

# set to True if you want to run that function
remove = False
rebuild = True
recalculate = True
recalculate_MC = True
revisualise = True


# Remove old results folder
if remove == True and os.path.exists('results'):
    shutil.rmtree('results')

# Set up models
models = []
models.append("bread") 
models.append('corn')

if rebuild == True:
    for model in models:
        write_database(model)
        add_uncertainties(model, dist_id=3, scale_percent=0.2) # see ids below
        inspect_db(model)
        export_db(model)

for model in models: 
    nodes, edges, model = extract_nodes_edges(model)
    write_process_diagram(nodes, edges, model)


 Writing database for: bread

Deleted old database fg_bread

****** Writing new database: fg_bread
Not able to determine geocollections for all datasets. This database is not ready for regionalization.
Title: Writing activities to SQLite3 database:
  Started: 05/22/2023 17:20:22
  Finished: 05/22/2023 17:20:22
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 3.36
{'depends': ['con391'], 'backend': 'sqlite', 'number': 5, 'modified': '2023-05-22T17:20:22.269898', 'geocollections': [], 'searchable': True, 'processed': '2023-05-22T17:20:22.311853', 'dirty': False}

   ACTIVITY:  
*****************************
Bread waste : kg : Bread waste
----------------------------
	TECHNOSPHERE EXCHANGES: 1
 *  Exchange: 0.0 kg 'Bread waste' (kg, None, None) to 'Bread waste' (kg, None, None)>
----------------------------
	BIOSPHERE EXCHANGES:  0
*****************************

   ACTIVITY:  
*****************************
Purification : kg : Purification
----------------------------
	TECHNOSPHER

In [10]:
if recalculate == True:
    for model in models:
        lca = get_LCA_scores(model)
        get_LCA_report(model)
        print(lca.score)
        print(lca.method)


***************** LCA calculations *****************

The functional unit is: 
Succinic acid production (bread), with an amount of 1 kg

*****
The exchanges of the functional unit are: 

Exchange: 1.0 kg 'Purification' (kg, None, None) to 'Succinic acid production (bread)' (kg, None, None)>


*****************
	 For the FU: 'Succinic acid production (bread)' 1 kg 
	 with the method '('IPCC 2013', 'climate change', 'global warming potential (GWP100)')' 
	the LCIA score is: -11.69 kg CO2-Eq


*****************
	 For the FU: 'Succinic acid production (bread)' 1 kg 
	 with the method '('IPCC 2013', 'climate change', 'global warming potential (GWP100)')' 
	the LCIA score is: -11.69 kg CO2-Eq
reports written to file
-11.691485616534877
('IPCC 2013', 'climate change', 'global warming potential (GWP100)')

***************** LCA calculations *****************

The functional unit is: 
Succinic acid production (corn), with an amount of 1 kg

*****
The exchanges of the functional unit are: 

Exc

In [11]:
if recalculate_MC == True:
    for model in models:
        single_score = get_LCA_scores(model)
        get_MCLCA_scores(model, single_score, iterations=1000)


***************** LCA calculations *****************

The functional unit is: 
Succinic acid production (bread), with an amount of 1 kg

*****
The exchanges of the functional unit are: 

Exchange: 1.0 kg 'Purification' (kg, None, None) to 'Succinic acid production (bread)' (kg, None, None)>


*****************
	 For the FU: 'Succinic acid production (bread)' 1 kg 
	 with the method '('IPCC 2013', 'climate change', 'global warming potential (GWP100)')' 
	the LCIA score is: -11.69 kg CO2-Eq

***************** Monte carlo - LCA calculations *****************



In [ ]:
import cowsay 
if revisualise == True:
    df = vis.plot_MC_results(distribution_type='Normal_1000')
    dic = df.describe().to_dict()

    results_list = []
    results_list.append("==== Results for Monte Carlo analysis ====")
    for key in dic.keys():
        results_list.append("\n***  "  + key + "kg CO2eq / kg  ***")
        for k, v in dic[key].items():
            results_list.append(f"{k}, {v}")
        print(cowsay.turtle('\n'.join(results_list)))
 
    # with open('results/MC_results_cowsay.txt', 'w') as f:
    #     f.write(x)

In [ ]:
# import bw2analyzer as ba
# from bw2analyzer import ContributionAnalysis 
# ContributionAnalysis().annotated_top_processes(lca)
# ContributionAnalysis().annotated_top_emissions(lca)

# lca.dicts.X.biosphere
# ContributionAnalysis().d3_treemap(lca.dict
                                 
# import bw2calc as bc
# from bw2calc import graph_traversal as gt

# lca.demand
# act = bd.get_node(id=lca.demand.keys[0])

# bc.utils.print_recursive_supply_chain(lca.activity)
# gt.AssumedDiagonalGraphTraversal().calculate(lca)
# gt.MultifunctionalGraphTraversal().calculate(lca)


# """
# STATS_ARRAYS DISTRIBUTION IDS
#  0: stats_arrays.distributions.undefined.UndefinedUncertainty,
#  1: stats_arrays.distributions.undefined.NoUncertainty,
#  2: stats_arrays.distributions.lognormal.LognormalUncertainty,
#  3: stats_arrays.distributions.normal.NormalUncertainty,
#  4: stats_arrays.distributions.geometric.UniformUncertainty,
#  5: stats_arrays.distributions.geometric.TriangularUncertainty,
#  6: stats_arrays.distributions.bernoulli.BernoulliUncertainty,
#  7: stats_arrays.distributions.discrete_uniform.DiscreteUniform,
#  8: stats_arrays.distributions.weibull.WeibullUncertainty,
#  9: stats_arrays.distributions.gamma.GammaUncertainty,
#  10: stats_arrays.distributions.beta.BetaUncertainty,
#  11: stats_arrays.distributions.extreme.GeneralizedExtremeValueUncertainty,
#  12: stats_arrays.distributions.student.StudentsTUncertaint
# """